# La ecuación de calor en 2D

Los sistemas físicos raremente viven en 1 sola dimensión. Es más usual que sean dos-dimensionales o tres-dimensionales. 

Dado que los sistemas 3D son computacionalmente demandantes, y las ideas a menudo son las mismas que en 2D, nos restringiremos aquí al caso 2D.

La ecuación de difusión en 2D es

$$\frac{\partial u(x,y,t)}{\partial t} = \nabla^2 u(x,y,t),$$

donde $\nabla^2 u(x,y,t) = \frac{\partial^2 u(x,y,t)}{\partial x^2} + \frac{\partial^2 u(x,y,t)}{\partial y^2}$.

Para discretizar el sistema, utilizaremos una malla $(x_i, y_j; t_n)$, con tamaños de paso $h$ para el tiempo, $k$ para $x$, y $l$ para $y$.

In [1]:
using Plots,Interact
gr()

Plots.GRBackend()

In [2]:
function malla(LX::Tuple{Float64,Float64},LY::Tuple{Float64,Float64},tf::Number;h::Number=1e-2,k::Number=1e-2,l::Number=1e-2)
    X=[LX[1]]
    x=LX[1]+k
    while x<LX[1]
        push!(X,x)
        x=x+k
    end
    Y=[LY[1]]
    y=LY[1]+l
    while y<LY[2]
        push!(Y,y)
        y=y+l
    end
    T=[0.0]
    t=0.0+h
    while t<tf
        push!(T,t)
        t=t+h
    end
    return (T,X,Y)
end

malla (generic function with 1 method)

In [90]:
collect(2.0:1e-2:4.0)

201-element Array{Float64,1}:
 2.0 
 2.01
 2.02
 2.03
 2.04
 2.05
 2.06
 2.07
 2.08
 2.09
 2.1 
 2.11
 2.12
 ⋮   
 3.89
 3.9 
 3.91
 3.92
 3.93
 3.94
 3.95
 3.96
 3.97
 3.98
 3.99
 4.0 

In [3]:
malla((-2.0,2.0),(-1.0,1.0),5.0,h=1e-2,k=.02,l=.03)

([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09  …  4.91,4.92,4.93,4.94,4.95,4.96,4.97,4.98,4.99,5.0],[-2.0,-1.98,-1.96,-1.94,-1.92,-1.9,-1.88,-1.86,-1.84,-1.82  …  1.8,1.82,1.84,1.86,1.88,1.9,1.92,1.94,1.96,1.98],[-1.0,-0.97,-0.94,-0.91,-0.88,-0.85,-0.82,-0.79,-0.76,-0.73  …  0.71,0.74,0.77,0.8,0.83,0.86,0.89,0.92,0.95,0.98])

**[1]** (i) Encuentra una discretización para $\nabla^2 u$ en el punto $(x_i, y_j; t_n)$.

(ii) Así, escribe la ecuación discretizada en la malla, y de ahí la regla para la evolución de Euler en el tiempo.

(iii) ¿Cómo son las condiciones de frontera de Dirichlet ahora? ¿Qué esperas ver físicamente para este tipo de condiciones absorbentes?

### [1]

Podemos discretizar la ecuación utilizando la misma discretización que la que se uso en una dimension, pero ahora aproximando la derivada respecto a cada variable. Es decir:

$$u(t+h,x,y)= u(t,x,y)+\frac{hD}{k^2}(u(t,x+k,y)+u(t,x-k,y)-2u(t,x,y)) +\frac{hD}{l^2}(u(t,x,y+l)+u(t,x,y-l)-2u(t,x,y))$$

Usando la notación anterior:

$$u^{n+1}_{j,i}= u^n_(j,i)+\frac{hD}{k^2}(u^n_{j+1,i}+u^n_{j-1,i}-2u^n_{j,i}) + \frac{hD}{l^2}(u^n_{j,i+1}+u^n_{j,i-1}-2u^n_{j,i})$$

Las condiciones de frontera deben ser ahora sobre todos las orillas de las funciones, es decir


$$u^n_{1,i}=u^n_{J,i}=u^n_{j,I} = u^n_{j,1} =0\quad \quad \forall i,j$$

**[2]** 

(i) Implementa esto computacionalmente y haz una animación para una condición inicial tipo delta. 
Ahora en lugar de un vector para representar al estado actual del sistema, necesitarás una *matriz* para las $u^n_{i,j}$ en el tiempo actual $t_n$.

(ii) ¿Qué ocurre para otras condiciones iniciales?

In [4]:
#[2]
function deltadir(X;inf::Float64=1e5)
    if norm(X)<=1/inf
        return inf
    else
        return 0.0
    end
end

deltadir (generic function with 1 method)

In [5]:
A=-1:.01:1
M=[deltadir([x,y],inf=1e2) for x in A, y in A]
display(surface(A,A,[50cos(pi/2*x)*cos(pi/2*y) for x in A, y in A]))
display(surface(A,A,[deltadir([x,y],inf=1e2) for x in A, y in A]))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,279.187 39.4926,279.187 33.587,283.103 39.4926,279.187 39.4926,266.684 36.5398,268.641 39.4926,266.684 39.4926,254.18 33.587,258.096 39.4926,254.18 
 39.4926,241.677 36.5398,243.634 39.4926,241.677 39.4926,229.173 33.587,233.089 39.4926,229.173 39.4926,216.67 36.5398,218.627 39.4926,216.67 39.4926,204.166 
 33.587,208.082 39.4926,204.166 39.4926,191.663 36.5398,193.62 39.4926,191.663 39.4926,179.159 33.587,183.075 39.4926,179.159 39.4926,166.656 36.5398,168.613 
 39.4926,166.656 39.4926,154.152 33.587,158.068 39.4926,154.152 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,279.187 39.4926,279.187 33.587,283.103 39.4926,279.187 73.2435,292.408 70.2907,294.365 73.2435,292.408 106.994,305.628 101.089,309.544 106.994,305.628 
 140.745,318.849 137.793,320.807 140.745,318.849 174.496,332.069 168.591,335.985 174.496,332.069 208.247,345.29 205.294,347.248 208.247,345.29 241.998,358.511 
 236.093,362.426 241.998,358.511 275.749,371.731 272.796,373.689 275.749,371.731 309.5,384.952 303.594,388.867 309.5,384.952 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 309.5,384.952 309.5,384.952 314.9,387.067 309.5,384.952 337.82,366.175 340.52,367.233 337.82,366.175 366.141,347.398 371.541,349.513 366.141,347.398 
 394.461,328.621 397.161,329.679 394.461,328.621 422.781,309.844 428.182,311.96 422.781,309.844 451.102,291.067 453.802,292.125 451.102,291.067 479.422,272.29 
 484.822,274.406 479.422,272.29 507.743,253.514 510.443,254.571 507.743,253.514 536.063,234.737 541.463,236.852 536.063,234.737 
 "/>

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#00002d; stroke-width:1; stroke-opacity:1; fill:none" points="
 17.9752,175.099 16.9051,175.091 16.5484,175.774 16.5484,176.597 16.9051,177.56 17.6185,178.251 19.0453,179.221 20.1154,180.052 20.8288,181.154 21.1855,182.117 
 21.1855,183.351 20.8288,184.034 20.4721,184.306 19.402,184.298 17.9752,183.739 16.9051,182.908 16.5484,182.357 16.1917,181.395 16.1917,180.16 16.5484,179.477 
 17.2618,178.934 18.3319,178.942 19.7587,179.089 20.4721,178.957 20.8288,178.274 20.8288,177.451 20.4721,176.488 19.402,175.658 17.9752,175.099 
 "/>
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,279.187 39.4926,279.187 33.587,283.103 39.4926,279.187 39.4926,266.684 36.5398,268.641 39.4926,266.684 39.4926,254.18 33.587,258.096 39.4926,254.18 
 39.4926,241.677 36.5398,243.634 39.4926,241.677 39.4926,229.173 33.587,233.089 39.4926,229.173 39.4926,216.67 36.5398,218.627 39.4926,216.67 39.4926,204.166 
 33.587,208.082 39.4926,204.166 39.4926,191.663 36.5398,193.62 39.4926,191.663 39.4926,179.159 33.587,183.075 39.4926,179.159 39.4926,166.656 36.5398,168.613 
 39.4926,166.656 39.4926,154.152 33.587,158.068 39.4926,154.152 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,279.187 39.4926,279.187 33.587,283.103 39.4926,279.187 73.2435,292.408 70.2907,294.365 73.2435,292.408 106.994,305.628 101.089,309.544 106.994,305.628 
 140.745,318.849 137.793,320.807 140.745,318.849 174.496,332.069 168.591,335.985 174.496,332.069 208.247,345.29 205.294,347.248 208.247,345.29 241.998,358.511 
 236.093,362.426 241.998,358.511 275.749,371.731 272.796,373.689 275.749,371.731 309.5,384.952 303.594,388.867 309.5,384.952 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 309.5,384.952 309.5,384.952 314.9,387.067 309.5,384.952 337.82,366.175 340.52,367.233 337.82,366.175 366.141,347.398 371.541,349.513 366.141,347.398 
 394.461,328.621 397.161,329.679 394.461,328.621 422.781,309.844 428.182,311.96 422.781,309.844 451.102,291.067 453.802,292.125 451.102,291.067 479.422,272.29 
 484.822,274.406 479.422,272.29 507.743,253.514 510.443,254.571 507.743,253.514 536.063,234.737 541.463,236.852 536.063,234.737 
 "/>
 
<image width="496" height="381" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAfAAAAF9CAYAAAD7gjrgAAAPuklEQVR4nO3da3LjuBWAUciVWUSy
gslmk01kYUllGR0C+UFRdtt6USIJXOCcqqn5Mek2M23rm3v5OpVSEgAQy0ftAwAA1hNwAAhIwAEg
IAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAE
HAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAA
CEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAg
AQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQc
AAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAI
SMABICABB4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICAB
B4CABBwAAhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwA
AhJwAAhIwAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhI
wAEgIAEHgIAEHAACEnAACEjAASAgAQeAgAQcAAIScAAISMABICABB4CABBwAAhJwAAhIwAEgIAEH
gIAEHBpxOv1Rah8DEMepFJ8ZUNO1cJfy61TjWIA4BBwqejR1Czlwi4BDJWtW5kIOfOccOBzsdPqj
rD3fvfyatb/2V/6X/0KHTgk4HGirC9WeDXnJ+g29EnA4yB5XmT8KeXaKDLr1l9oHAD076taw719n
OWc+TTmlP444AuBoJnDYSc37upfJPFuhQ7cEHHbQykNZ3GUC/bJChw21Eu7FNOXfjsntaNAPAYcN
tBbuxfcV+nKcQg7xWaHDm1qNd0q3V+iv3IsOtMUEDi+KEMBpynf/+a2r14H2mcDhBRHindLPFfoj
JnOIQ8BhpUiBe/VJbEIO7bNChydEjNm///uPhyv0R6zYoV0mcHggYrxTmqfvXEr6z3//udnvaTKH
dpjA4Y7IscqlpI+cUtlhZnY7GtQn4HBD5HindJ7AP1I67fj/QsihHgGHb6KHezFNJZ1KSae0f1ud
K4fjCTic9R

In [27]:
function calorabs(ut0::Function, LX::Tuple{Float64,Float64},LY::Tuple{Float64,Float64}, tf::Float64; d::Number=1, h::Number=1e-2, k::Number=1e-2,l::Number=1e-2, test=false)
    println((h*d/(k^2),h*d/(l^2)))
    (T,X,Y)=malla(LX,LY,tf,h=h,k=k,l=l)
    
    u0=[ut0([x,y]) for x in X, y in Y]
    u0=u0'
    total=[u0]
    for t in 1:(length(T)-1)
        B=total[t]
        CX=[B[i,j] for i in 2:size(B)[1],j in 1:size(B)[2]]
        CX=CX'
        CX=hcat(CX,zeros(size(B)[2]))
        
        DX=[B[i,j] for i in 1:(size(B)[1]-1),j in 1:size(B)[2]]
        DX=DX'
        DX=hcat(zeros(size(B)[2]),DX)
        CY=[B[i,j] for i in 1:size(B)[1],j in 2:size(B)[2]]
        CY=CY'
        CY=vcat(CY,zeros(size(B)[1])')
        DY=[B[i,j] for i in 1:size(B)[1],j in 1:(size(B)[2]-1)]
        DY=DY'
        DY=vcat(zeros(size(B)[1])',DY)
        u=B+(h*d/(k^2))*(CX+DX-2*B)+(h*d/(l^2))*(CY+DY-2*B)
        if test==true
            @show i
            @show B
            @show CX
            @show DX
            @show CY
            @show DY
            @show u
        end
        u[:,end]=0.0
        u[end,:]=0.0
        u[1,:]=0.0
        u[:,1]=0.0
        push!(total,u)
    end
    return total
end

calorabs (generic function with 1 method)

, Tuple{Float64, Float64}, Tuple{Float64, Float64}, Float64) in module Main at In[25]:2 overwritten at In[27]:2.


In [85]:
Aldo=calorabs(x->deltadir(x,inf=10.0),(-1.0,1.0),(-1.0,1.0),10.0,d=.0001,h=.1,k=0.01,l=0.01)

(0.1,0.1)


101-element Array{Array{Float64,2},1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0

In [86]:
Aldo[end]

200×200 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [87]:
@manipulate for i in 1:length(Aldo)
    surface(Aldo[i][75:125,75:125])
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(51, nactions=1),"i",51,"51",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(68=>"68",2=>"2",89=>"89",11=>"11",39=>"39",46=>"46",85=>"85",25=>"25",55=>"55",42=>"42"…)),Any[],Any[],true,"horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#00002d; stroke-width:1; stroke-opacity:1; fill:none" points="
 6.68429,178.161 5.60444,178.164 5.24449,178.851 5.24449,179.674 5.60444,180.633 6.32434,181.317 7.76414,182.273 8.84399,183.092 9.56389,184.188 9.92384,185.146 
 9.92384,186.381 9.56389,187.068 9.20394,187.343 8.12409,187.346 6.68429,186.801 5.60444,185.982 5.24449,185.434 4.88453,184.475 4.88453,183.241 5.24449,182.554 
 5.96439,182.003 7.04424,182 8.48404,182.133 9.20394,181.994 9.56389,181.308 9.56389,180.485 9.20394,179.526 8.12409,178.706 6.68429,178.161 
 "/>
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,279.187 21.7148,279.187 15.3788,283.243 21.7148,279.187 21.7148,266.684 18.5468,268.711 21.7148,266.684 21.7148,254.18 15.3788,258.236 21.7148,254.18 
 21.7148,241.677 18.5468,243.704 21.7148,241.677 21.7148,229.173 15.3788,233.229 21.7148,229.173 21.7148,216.67 18.5468,218.697 21.7148,216.67 21.7148,204.166 
 15.3788,208.222 21.7148,204.166 21.7148,191.663 18.5468,193.691 21.7148,191.663 21.7148,179.159 15.3788,183.215 21.7148,179.159 21.7148,166.656 18.5468,168.684 
 21.7148,166.656 21.7148,154.152 15.3788,158.208 21.7148,154.152 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,279.187 44.0887,287.648 40.9207,289.676 44.0887,287.648 72.0561,298.225 65.7202,302.28 72.0561,298.225 100.023,308.801 96.8555,310.829 100.023,308.801 
 127.991,319.378 121.655,323.433 127.991,319.378 155.958,329.954 152.79,331.982 155.958,329.954 183.926,340.531 177.59,344.586 183.926,340.531 211.893,351.107 
 208.725,353.135 211.893,351.107 239.86,361.684 233.525,365.739 239.86,361.684 267.828,372.26 264.66,374.288 267.828,372.26 295.795,382.837 289.459,386.892 
 295.795,382.837 301.389,384.952 
 "/>
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 301.389,384.952 320.163,372.935 322.959,373.992 320.163,372.935 343.63,357.913 349.224,360.028 343.63,357.913 367.098,342.892 369.894,343.949 367.098,342.892 
 390.565,327.87 396.158,329.985 390.565,327.87 414.032,312.849 416.829,313.906 414.032,312.849 437.5,297.827 443.093,299.942 437.5,297.827 460.967,282.806 
 463.764,283.863 460.967,282.806 484.435,267.784 490.028,269.899 484.435,267.784 507.902,252.762 510.699,253.82 507.902,252.762 531.37,237.741 536.963,239.856 
 531.37,237.741 536.063,234.737 
 "/>

In [88]:
@gif for i in 1:length(Aldo)
    surface(Aldo[i][75:125,75:125])
end

INFO: Saved animation to C:\Users\Aldo\Google Drive\Materias Facultad\Fis Comp\FisicaComputacional2017_2\notebooks\tmp.gif


Plots.AnimatedGif("C:\\Users\\Aldo\\Google Drive\\Materias Facultad\\Fis Comp\\FisicaComputacional2017_2\\notebooks\\tmp.gif")

In [14]:
vcat(zeros(6)',M2)

LoadError: ArgumentError: number of columns of each array must match (got (6,201))

In [15]:
M[1,:]=0

0

In [16]:
M[end,end]

0.0

In [17]:
M=[a*b for a in 1:6,b in 1:3]
M=M'
heatmap(M)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
<image width="505" height="381" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAfkAAAF9CAYAAAAHlxEqAAAGl0lEQVR4nO3XMUocABRF0ZmgE0Fh
GIgiqAFJYaNlqiSkzwLcsBuws7BVLCSFWKUIk01M+HA5ZwWP31z+8vrwdrtgp34enU5PSPl+8jY9
IeXrl8fpCSkXP+6nJ6T8/fVtekLKh+kBAMD/IfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXy
ABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIA
ECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQ
JfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl
8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXy
ABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIA
ECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQ
JfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl
8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXy
ABAl8gAQJfIAECXyABC197p4mt6Q8/C+mZ4Qs54ekPL7z830hJSrl7PpCSmXd8/TE1J88gAQJfIA
ECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQ
JfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl
8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXy
ABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIA
ECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQ
JfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl
8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXy
ABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIA
ECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABAl8gAQJfIAECXyABC1PFh93k6PqFnvn09PSDle
uOcunWw30xNSjler6Qkpnz4upyek+OQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEg
SuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK
5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErk
ASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQB
IErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEg
SuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK
5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErk
ASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQB
IErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEgSuQBIErkASBK5AEg
SuQBIErkASBK5AEg6h8ApRW5q1btpgAAAABJRU5ErkJggg==
" transform="translate(30, 3)"/>

**[3]** (i) Repite la pregunta [2] con condiciones de Neumann.

(ii) Ahora hazlo con condiciones **periódicas** de frontera. Aquí, el vecino de una celda en la frontera que "se sale del sistema" se toma como la celda *del otro lado del sistema*. [Esto da la topología de un toro al sistema, y minimiza el efecto de las fronteras, para simular mejor un sistema de tamaño "infinito".]